In [7]:
import os
import pandas as pd

# Set directory where CSV files are located
dataSourceFolder = "C:\\Users\\e0253700\\Desktop\\NUS\\MFE\\Semester 2\\FE5107\\Project\\TRD_Dalyr\\"

# Get a list of all the CSV files in the folder
csv_files = [file for file in os.listdir(dataSourceFolder) if file.endswith('.csv')]

# Create an empty list to store the dataframe
dfs = []

# Loop through the CSV files and read them into dataframes
for file in csv_files:
    file_path = os.path.join(dataSourceFolder, file)
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate the dataframes into a single dataframe
df = pd.concat(dfs, ignore_index=True)

print(df.shape)

(15098260, 24)


In [2]:
# df.to_csv(dataSourceFolder + "agg.csv", index=False)

In [8]:
# Data cleaning

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Keep Markettype = 1, 4, 16, 32
valid_market_types = [1, 4, 16, 32]
df = df[df['Markettype'].isin(valid_market_types)]

# Keep only useful columns
columns_to_keep = ['Stkcd', 'Trddt', 'Clsprc', 'Dnshrtrd', 'Dsmvosd']
df = df.loc[:, columns_to_keep]

print(df.shape)

(4925100, 5)


In [9]:
# Create column for Daily Circulating Shares
df['Dcircshare'] = df['Dsmvosd'] / df['Clsprc']
df['year'] = pd.to_datetime(df['Trddt']).dt.year
df['month'] = pd.to_datetime(df['Trddt']).dt.month
print(df.shape)

(4925100, 8)


In [10]:
# Dataframe for results
liquidityColumns = ['Stkcd', 'year', 'month', 'STOM']
liquidity = pd.DataFrame(columns=liquidityColumns)

In [11]:
# Calculate STOM
import math
# Group the dataframe by the Stkcd 
groupByStock = df.groupby('Stkcd')

# Loop over stocks
for stockCode, stock in groupByStock:
    groupByYear = stock.groupby('year')
    # Loop over years
    for yearNum, year in groupByYear:
        groupByMonth = year.groupby('month')
        # Loop over months
        for monthNum, month in groupByMonth:
            STOD_sum = (month['Dnshrtrd'] / month['Dcircshare']).sum()
            newRecord = {'Stkcd': stockCode, 'year': yearNum, 'month': monthNum, 'STOM': math.log(STOD_sum)}
            newRecordDF = pd.DataFrame([newRecord])
            liquidity = pd.concat([liquidity, newRecordDF], ignore_index=True)
    #         break
    #     break
    # break

print(liquidity.shape)


(249483, 4)


In [14]:
print(liquidity.shape)
# print(liquidity)
# liquidity.to_csv(dataSourceFolder + "STOM.csv", index=False)

(249483, 4)


In [22]:
import numpy as np

liquidity['expSTOM'] = liquidity['STOM'].apply(np.exp)

In [42]:
# Create a function to calculate quarter averages
def calculate_quarter_average(year, months):
    if set(months).issubset(set(year['month'])):
        subset = year.loc[year['month'].isin(months), 'expSTOM']
        return np.mean(subset)
    else:
        return np.nan

In [50]:
# Calculate STOQ averages for each quarter
quarter_averages = {}
groupByStock = liquidity.groupby('Stkcd')
for stockCode, stock in groupByStock:
    groupByYear = stock.groupby('year')
    for yearNum, year in groupByYear:
        quarter_averages[(stockCode, yearNum, 1)] = math.log(calculate_quarter_average(year, [1, 2, 3]))
        quarter_averages[(stockCode, yearNum, 2)] = math.log(calculate_quarter_average(year, [4, 5, 6]))
        quarter_averages[(stockCode, yearNum, 3)] = math.log(calculate_quarter_average(year, [7, 8, 9]))
        quarter_averages[(stockCode, yearNum, 4)] = math.log(calculate_quarter_average(year, [10, 11, 12]))

# Map quarter averages to STOQ column
liquidity['STOQ'] = liquidity.apply(lambda row: quarter_averages.get((row['Stkcd'], row['year'], (row['month']-1)//3+1)), axis=1)

In [51]:
liquidity.to_csv(dataSourceFolder + "STOQ.csv", index=False)

In [66]:
# Calculate yearly averages
def calculate_yearly_average(year):
    return year['expSTOM'].mean()

# Calculate yearly averages for each stock
yearly_averages = {}
groupByStock = liquidity.groupby('Stkcd')
for stockCode, stock in groupByStock:
    groupByYear = stock.groupby('year')
    for yearNum, year in groupByYear:
        yearly_averages[(stockCode, yearNum)] = math.log(calculate_yearly_average(year))

# Map yearly averages to STOA column
liquidity['STOA'] = liquidity.apply(lambda row: yearly_averages.get((row['Stkcd'], row['year'])), axis=1)

In [67]:
liquidity.to_csv(dataSourceFolder + "Liquidity.csv", index=False)